# 解壓縮

In [8]:
!rm -rf dataset_xy
!rm -rf output
!unzip -q road_following_dataset_xy_2024-11-03_12-08-48.zip

# 函式庫

In [9]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn.functional as F
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn as nn


# 檔案格式解碼


In [10]:
def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):

    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))

        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x

        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

        return image, torch.tensor([x, y]).float()

dataset = XYDataset('dataset_xy', random_hflips=False)

# 區分訓練集


In [12]:
test_percent = 0.2
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

# 提早結束


In [13]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


# 訓練


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset


# 將資料集切分為訓練集和驗證集
full_dataset = XYDataset('dataset_xy', random_hflips=False)

save_path = "output"

# 訓練與驗證函數更新
def train_and_validate_model(model, model_name, save_path):
    learning_rate = 1e-5
    NUM_EPOCHS = 30
    best_loss = 1e9

    model.fc = nn.Linear(512, 2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)  # 設置 Early Stopping

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        for images, labels in iter(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.mse_loss(outputs, labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in iter(test_loader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = F.mse_loss(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(test_loader)

        print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        if val_loss < best_loss:
            #torch.save(model.state_dict(), f'{save_path}/{model_name}_best.pth')
            best_loss = val_loss

        # 檢查 Early Stopping 條件
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

    # 最終模型存檔
    final_model_path = f'{save_path}/{model_name}_final.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f'{model_name} 已存檔至 {final_model_path}')


# 修改 `train_and_save_model` 的呼叫來傳遞新的 `train_loader` 和 `val_loader`
resnet50 = models.resnet50(pretrained=True)
resnet18 = models.resnet18(pretrained=True)
resnet34 = models.resnet34(pretrained=True)
train_and_validate_model(resnet34, 'resnet34', save_path)
train_and_validate_model(resnet50, 'resnet50', save_path)
train_and_validate_model(resnet18, 'resnet18', save_path)

<ipython-input-10-fcb5603b4281>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:


Epoch 1/30, Train Loss: 0.3722, Val Loss: 0.1090
Epoch 2/30, Train Loss: 0.0727, Val Loss: 0.0592
Epoch 3/30, Train Loss: 0.0563, Val Loss: 0.0520
Epoch 4/30, Train Loss: 0.0461, Val Loss: 0.0519
Epoch 5/30, Train Loss: 0.0414, Val Loss: 0.0415
Epoch 6/30, Train Loss: 0.0378, Val Loss: 0.0366


# TensorRT


## 安裝


In [ ]:
!rm -rf build/
!rm -rf dist/
!rm -rf torch2trt.egg-info/
!rm -rf torch2trt/

In [ ]:
!dpkg -l | grep nvinfer
!sudo apt-get update
!sudo apt-get install -y tensorrt python3-libnvinfer-dev
!sudo apt-get update
!sudo apt-get install -y libnvinfer8 libnvinfer-plugin8 libnvparsers8 libnvonnxparsers8 libnvinfer-dev python3-libnvinfer
!sudo apt-get install python3-libnvinfer
!python3 -m pip install --upgrade tensorrt

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,442 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
import tensorrt
print(tensorrt.__version__)
assert tensorrt.Builder(tensorrt.Logger())

ModuleNotFoundError: No module named 'tensorrt'

In [ ]:
!pip3 install git+https://github.com/NVIDIA-AI-IOT/torch2trt.git --target=/usr/lib/python3/dist-packages/


In [ ]:
try:
    from torch2trt import torch2trt
    print("torch2trt has been successfully imported.")
except ModuleNotFoundError as e:
    print("Module not found:", e)


In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()
model.load_state_dict(torch.load('output/resnet18_lr1e-05_epochs40_batch16.pth'))
device = torch.device('cuda')
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)
torch.save(model_trt.state_dict(), 'output/resnet18_lr1e-05_epochs40_batch16_trt.pth')

<ipython-input-12-9a1d6db45d19>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('output/resnet18_lr1e-05_epochs40_batch16.pth'))


ModuleNotFoundError: No module named 'torch2trt'

In [ ]:
# Compress the 'output' folder into a zip file in Colab
!zip -r output.zip output/


  adding: output/ (stored 0%)
  adding: output/resnet50_lr1e-05_epochs50_batch16.pth (deflated 7%)
  adding: output/resnet50_lr1e-05_epochs50_batch16_trt_fp32.pth (deflated 26%)
  adding: output/resnet18_lr1e-05_epochs50_batch16.pth (deflated 7%)
  adding: output/resnet34_lr1e-05_epochs50_batch16.pth (deflated 7%)
  adding: output/resnet18_lr1e-05_epochs50_batch16_trt_fp8.pth (deflated 27%)
  adding: output/resnet50_lr1e-05_epochs50_batch16_trt_fp16.pth (deflated 27%)
  adding: output/resnet18_lr1e-05_epochs50_batch16_trt_fp16.pth (deflated 27%)
  adding: output/.ipynb_checkpoints/ (stored 0%)
  adding: output/resnet34_lr1e-05_epochs50_batch16_trt_fp8.pth (deflated 27%)
  adding: output/resnet50_lr1e-05_epochs50_batch16_trt_fp8.pth (deflated 27%)
  adding: output/resnet34_lr1e-05_epochs50_batch16_trt_fp16.pth (deflated 27%)
  adding: output/resnet34_lr1e-05_epochs50_batch16_trt_fp32.pth (deflated 26%)
  adding: output/resnet18_lr1e-05_epochs50_batch16_trt_fp32.pth (deflated 26%)
